# EDA Completo: Eficiencia de Combustible en Automóviles

## Objetivos del Proyecto
Realizar un Análisis Exploratorio de Datos (EDA) exhaustivo para entender qué factores influyen en el consumo de gasolina de un vehículo.

**Hitos del análisis:**
1.  **Limpieza:** Detección y manejo de datos faltantes.
2.  **Estadística Descriptiva:** Análisis de tendencias centrales (media, mediana) y dispersión.
3.  **Visualización Univariable:** Distribución de potencia (`horsepower`) y rendimiento (`mpg`).
4.  **Análisis Bivariable:** Relación entre peso y consumo, y diferencias por país de origen.
5.  **Detección de Outliers:** Identificación de vehículos con características inusuales.

## 1. Carga y Visión General
Cargamos el dataset `mpg` y hacemos una inspección inicial de su estructura y tipos de datos.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Configuración visual profesional
sns.set_theme(style="whitegrid", context="notebook")

# Carga del dataset
df = sns.load_dataset('mpg')

print(f"Dimensiones del dataset: {df.shape}")
print("\n--- Primeras 5 filas ---")
display(df.head())

print("\n--- Información General ---")
print(df.info())

## 2. Limpieza de Datos
Al revisar `df.info()`, observamos que la columna `horsepower` tiene algunos valores nulos (el conteo no-null es menor que el total).
* **Acción:** Eliminaremos esas filas, ya que son pocas y no afectarán significativamente al análisis general, pero asegurarían que nuestros cálculos estadísticos sean correctos.

In [ ]:
# Ver cuántos nulos hay
nulos = df['horsepower'].isnull().sum()
print(f"Valores nulos detectados en 'horsepower': {nulos}")

# Eliminación de filas con nulos
df_clean = df.dropna()

print(f"Dimensiones tras limpieza: {df_clean.shape}")

## 3. Estadística Descriptiva (Variables Numéricas)
Analizaremos las estadísticas clave de nuestras variables principales:
* `mpg` (Millas por galón - Eficiencia)
* `horsepower` (Potencia)
* `weight` (Peso)

In [ ]:
# Resumen estadístico transpuesto para mejor lectura
display(df_clean[['mpg', 'horsepower', 'weight', 'acceleration']].describe().T)

### Interpretación Preliminar
* **Rendimiento (`mpg`):** El promedio es de ~23.4 mpg. El rango va desde 9 mpg (muy ineficiente) hasta 46.6 mpg (muy eficiente).
* **Potencia (`horsepower`):** Hay una gran dispersión. La media es 104 hp, pero el máximo llega a 230 hp. La mediana (93.5 hp) es menor que la media, lo que sugiere un **sesgo a la derecha** (hay pocos autos muy potentes que suben el promedio).

## 4. Visualización Univariable
¿Cómo se distribuye la potencia de los autos? Usaremos un Histograma con curva de densidad.

In [ ]:
plt.figure(figsize=(10, 6))

sns.histplot(df_clean['horsepower'], kde=True, color='firebrick', bins=20)

plt.title('Distribución de Potencia (Horsepower)')
plt.xlabel('Caballos de Fuerza (hp)')
plt.ylabel('Frecuencia')

# Líneas de referencia
plt.axvline(df_clean['horsepower'].mean(), color='blue', linestyle='--', label='Media')
plt.axvline(df_clean['horsepower'].median(), color='green', linestyle='-', label='Mediana')
plt.legend()

plt.show()

## 5. Análisis Categórico: Origen del Vehículo
¿Qué región domina el mercado en este dataset?

In [ ]:
plt.figure(figsize=(8, 5))

# Ordenar por cantidad
orden = df_clean['origin'].value_counts().index

sns.countplot(data=df_clean, x='origin', order=orden, palette='viridis')

plt.title('Cantidad de Vehículos por Origen')
plt.xlabel('País / Región')
plt.ylabel('Cantidad')
plt.show()

## 6. Relación Multivariable: Potencia vs Rendimiento vs Origen
¿Es cierto que los autos más potentes gastan más gasolina? ¿Cómo afecta el país de origen a esta relación?

In [ ]:
plt.figure(figsize=(10, 6))

sns.scatterplot(
    data=df_clean,
    x='horsepower',
    y='mpg',
    hue='origin',   # Color por país
    size='weight',  # Tamaño por peso del auto
    sizes=(20, 200),
    alpha=0.7
)

plt.title('Relación: Potencia vs Rendimiento (por Origen y Peso)')
plt.xlabel('Potencia (hp)')
plt.ylabel('Millas por Galón (mpg)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

### Observaciones importantes
1.  **Correlación Negativa Clara:** A mayor potencia (`horsepower`), menor rendimiento (`mpg`). La nube de puntos desciende de izquierda a derecha.
2.  **El Factor Peso:** Los puntos más grandes (autos pesados) se concentran en la zona de baja eficiencia y alta potencia.
3.  **Patrón Regional:**
    * **USA (Azul):** Domina el sector de autos potentes, pesados y de alto consumo.
    * **Japón y Europa:** Se agrupan en la zona de alta eficiencia (alto mpg), menor peso y menor potencia.

## 7. Detección de Outliers en Aceleración
Analizaremos la variable `acceleration` (tiempo en segundos para alcanzar cierta velocidad). Usaremos el método del **Rango Intercuartil (IQR)** para ver si hay autos inusualmente lentos o rápidos.

In [ ]:
# 1. Boxplot para inspección visual
plt.figure(figsize=(10, 4))
sns.boxplot(x=df_clean['acceleration'], color='orange')
plt.title('Boxplot de Aceleración (Detectando Outliers)')
plt.show()

# 2. Cálculo matemático
Q1 = df_clean['acceleration'].quantile(0.25)
Q3 = df_clean['acceleration'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# 3. Filtrar los outliers
autos_outliers = df_clean[
    (df_clean['acceleration'] < limite_inferior) |
    (df_clean['acceleration'] > limite_superior)
]

print(f"Rango normal de aceleración: {limite_inferior:.2f}s a {limite_superior:.2f}s")
print(f"Número de outliers detectados: {len(autos_outliers)}")

print("\n--- Autos con aceleración inusual ---")
display(autos_outliers[['name', 'origin', 'acceleration', 'horsepower']].sort_values('acceleration'))

## Conclusiones del EDA

Tras analizar el dataset `mpg`, podemos concluir:

1.  **Eficiencia vs Potencia:** Existe una relación inversa fuerte. Si buscas eficiencia (`mpg` > 30), debes buscar autos con menos de 100 caballos de fuerza.
2.  **Mercado Americano vs Internacional:**
    * Los autos de **EE.UU.** en esta época se caracterizan por ser "Muscle Cars": pesados, muy potentes, pero ineficientes.
    * Los autos de **Japón y Europa** optimizan el consumo, manteniéndose ligeros y con motores más pequeños.
3.  **Anomalías (Outliers):** Detectamos vehículos atípicos en aceleración. Por ejemplo, el *Peugeot 504* es inusualmente lento (24.8s), mientras que algunos deportivos son extremadamente rápidos para el promedio de la época.
4.  **Recomendación de Datos:** Para futuros modelos predictivos de consumo de gasolina, las variables `weight` y `horsepower` serán los predictores más fuertes, mientras que `origin` servirá como una excelente variable categórica de segmentación.